In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import chargenet_trafo, chargenet_trafo_new

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [4]:
data1 = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols')

data2 = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols')

In [5]:
charge_input = tf.keras.Input(shape=(1,))
params_input = tf.keras.Input(shape=(len(data1.labels),))
t = chargenet_trafo_new(labels=data1.labels)

h = t(charge_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
#h = tf.keras.layers.Dense(512, activation="relu")(h)
#h = tf.keras.layers.Dropout(0.001)(h)
#h = tf.keras.layers.Dense(256, activation="relu")(h)
#h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

charge_net = tf.keras.Model(inputs=[charge_input, params_input], outputs=outputs)
charge_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
chargenet_trafo_new (chargenet_ (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           320         chargenet_trafo_new[0][0]        
______________________________________________________________________________________________

In [12]:
optimizer = tf.keras.optimizers.Adam(0.00001)
charge_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [7]:
train_id = 'ChargeNet_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [8]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i', ))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [9]:
train_data1, test_data1 = data1.get_chargenet_data(train_batch_size=4096,
                                                 test_batch_size=4096,
                                                )
train_data2, test_data2 = data2.get_chargenet_data(train_batch_size=4096,
                                                 test_batch_size=4096,
                                                )

In [10]:
train_data = tf.data.Dataset.concatenate(train_data1,train_data2)
test_data = tf.data.Dataset.concatenate(test_data1,test_data2)

In [ ]:
hist = charge_net.fit(train_data, epochs=600, verbose=2, callbacks=callbacks, validation_data=test_data,
                     initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 401/600
988/988 - 18s - loss: 0.4537 - val_loss: 0.4735
Epoch 402/600
988/988 - 18s - loss: 0.4534 - val_loss: 0.4763
Epoch 403/600
988/988 - 17s - loss: 0.4531 - val_loss: 0.4736
Epoch 404/600
988/988 - 19s - loss: 0.4530 - val_loss: 0.4730
Epoch 405/600
988/988 - 17s - loss: 0.4533 - val_loss: 0.4766
Epoch 406/600
988/988 - 17s - loss: 0.4529 - val_loss: 0.4712
Epoch 407/600
988/988 - 16s - loss: 0.4529 - val_loss: 0.4685
Epoch 408/600
988/988 - 18s - loss: 0.4528 - val_loss: 0.4745
Epoch 409/600
988/988 - 19s - loss: 0.4530 - val_loss: 0.4746
Epoch 410/600
988/988 - 18s - loss: 0.4530 - val_loss: 0.4727
Epoch 411/600
988/988 - 16s - loss: 0.4528 - val_loss: 0.4762
Epoch 412/600
988/988 - 16s - loss: 0.4531 - val_loss: 0.4762
Epoch 413/600
988/988 - 18s - loss: 0.4527 - val_loss: 0.4716
Epoch 414/600
988/988 - 18s - loss: 0.4529 - val_loss: 0.4718
Epoch 415/600


In [ ]:
# best valid loss L7 0.53....52
# L3: 0.48 ish
# L3 bigger net: 0.48...0.47 with 12+14,
# after epoch 400 reduced lr to 0.00001